In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy
import gensim
import logging

from common import create_engine
from common import display_all
from common import figsize
from common import save_df, load_df
from common import save_session, load_session

from util import show_importances
from util import split_X_y_all, split_X_y, split_data
from util import empty_features, column_feature, str_contains

from pbar import Pbar

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters() # converters e.g. for datetime in plots


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

ImportError: cannot import name 'show_importances' from 'util' (C:\Users\j\IdeaProjects\fiit_masters_thesis_project\jupyter\util.py)

In [5]:
SESSION_FILE_NAME = 'method-impl.session-db'

load_session(SESSION_FILE_NAME)

In [6]:
RANDOM_STATE = 123
np_random = np.random.RandomState(RANDOM_STATE)

### Load dataset

In [109]:
df = load_df('final_data.pickle')

In [110]:
df.head()

,id,url,title,perex,body,published_at,extracted_at,source_id,category,other_info,...,source_id,source_name,source_url,source_type,source_is_reliable,fb_sync_date,fb_reaction_count,fb_comment_count,fb_share_count,fb_popularity
0,235048,https://www.naturalnewsblogs.com/the-secret-to...,The Secret to Happiness &#8211; Revealed!,<p>&#8220;Why are you so happy all of the time...,“Why are you so happy all of the time?”\nI am ...,2013-08-07 12:36:20,2019-09-05 06:45:28.429274,142,"[Health, Mental Health]","{'tags': ['choices', 'depression', 'happiness'...",...,142,naturalnewsblogs.com,http://naturalnewsblogs.com,news_website,0,2019-10-14 15:29:44.601408,0,0,0,0
1,235036,https://www.naturalnewsblogs.com/us-government...,US government claims 100% ownership over all y...,<p>(NaturalNews) The United States government ...,(NaturalNews) The United States government cla...,2013-05-19 18:50:37,2019-09-05 06:45:27.633650,142,"[Health, News, Science, Weird, Biotechnology, ...","{'tags': ['gene patents', 'genetic slavery', '...",...,142,naturalnewsblogs.com,http://naturalnewsblogs.com,news_website,0,2019-10-14 15:29:44.601408,0,0,0,0
2,235039,https://www.naturalnewsblogs.com/angelina-joli...,Angelina Jolie copied by men! Surgeons now cut...,<p>(NaturalNews) Beyond merely inspiring women...,(NaturalNews) Beyond merely inspiring women to...,2013-05-20 18:54:30,2019-09-05 06:45:28.083048,142,"[Health, Science, Weird, Celebrity, Hospitals ...","{'tags': ['Angelina Jolie', 'BRCA genes', 'pro...",...,142,naturalnewsblogs.com,http://naturalnewsblogs.com,news_website,0,2019-10-14 15:29:44.601408,0,0,0,0
3,235041,https://www.naturalnewsblogs.com/natural-news-...,Natural News releases latest laboratory test r...,"<p>(NaturalNews) As Natural News readers know,...","(NaturalNews) As Natural News readers know, we...",2013-05-23 19:47:30,2019-09-05 06:45:28.167110,142,"[Health, Food, Nutrition]","{'tags': ['clean chlorella', 'heavy metals', '...",...,142,naturalnewsblogs.com,http://naturalnewsblogs.com,news_website,0,2019-10-14 15:29:44.601408,0,0,0,0
4,235052,https://www.naturalnewsblogs.com/food-insecuri...,Food Insecurity: A Solution Grows Under Your Feet,<p>Food insecurity is a serious problem that p...,Food insecurity is a serious problem that plag...,2013-08-08 10:12:31,2019-09-05 06:45:28.607943,142,"[Health, Nutritional Medicine, Food, Nutrition]","{'tags': ['broadleaf plantain', 'food insecuri...",...,142,naturalnewsblogs.com,http://naturalnewsblogs.com,news_website,0,2019-10-14 15:29:44.601408,0,0,0,0


In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164450 entries, 0 to 171743
Data columns (total 23 columns):
id                    164450 non-null int64
url                   164450 non-null object
title                 164450 non-null object
perex                 140260 non-null object
body                  164221 non-null object
published_at          164450 non-null datetime64[ns]
extracted_at          164450 non-null datetime64[ns]
source_id             164450 non-null int64
category              136428 non-null object
other_info            164446 non-null object
image_count           164450 non-null int64
video_count           164450 non-null int64
author_name           164450 non-null object
source_id             164450 non-null int64
source_name           164450 non-null object
source_url            164450 non-null object
source_type           164450 non-null object
source_is_reliable    164450 non-null int64
fb_sync_date          164450 non-null datetime64[ns]
fb_reaction_coun

-----

Rozdelenie hodnot popularity do 5 skupin

- `0 - 0.5`
- `0.5 - 0.75`
- `0.75 - 0.9`
- `0.9 - 0.95`
- `0.95 - 1`

In [249]:
def add_labels(df, quantiles, column='fb_popularity'):
    df = df.copy()
    label_str = f'{column}_label'
    
    df[label_str] = -1
    
    label = 1    
    for i in range(len(quantiles) - 1):
        low = df[column].quantile(quantiles[i])
        high = df[column].quantile(quantiles[i + 1])
        
        df.loc[(low <= df[column]) & (df[column] <= high), label_str] = int(label)
        
        label += 1
    df = df.drop(columns=[column])    
    return df.copy()

In [250]:
quantiles = [
    0,
    .50,
    .75,
    .90,
    .95,
    1
]

cols = [
    'fb_reaction_count',
    'fb_comment_count',
    'fb_share_count',
    'fb_popularity'    
]

for i in cols:
    print(df[i].quantile(quantiles))
    df = add_labels(df, quantiles, column=i)

0.00          0.0
0.50          0.0
0.75         53.0
0.90        460.0
0.95       1372.0
1.00    3929532.0
Name: fb_reaction_count, dtype: float64
0.00         0.0
0.50         0.0
0.75         8.0
0.90        92.0
0.95       311.0
1.00    797490.0
Name: fb_comment_count, dtype: float64
0.00         0.0
0.50         1.0
0.75        35.0
0.90       194.0
0.95       530.0
1.00    572708.0
Name: fb_share_count, dtype: float64
0.00          0.0
0.50          2.0
0.75        105.0
0.90        772.0
0.95       2257.0
1.00    5197586.0
Name: fb_popularity, dtype: float64


Pri jednotlivych zlozkach sme pri tomto rozdeleni nasli len 4 skupiny (lebo 1 == 2)

---

Jednoducha heuristika: ak je zdroj nedoveryhodny tak aj clanok je nedoveryhodny

In [251]:
df['is_fake_news_label'] = df.source_is_reliable.replace({0:1, 1:0})

---

### data cleanup

In [115]:
# clear body, perex, etc from html....

In [116]:
# drop rows without body
df = df[~df.body.isnull()]
df = df[~df.title.isnull()]

In [117]:
from bs4 import BeautifulSoup
import unicodedata

def clear_text(text):
    if text is None:
        return ''

    text = BeautifulSoup(text, features='html.parser').text
    text = text.lower()
    text = text.replace('\r', '')
    text = text.replace('\n', ' ')
    text = unicodedata.normalize('NFKD', text)

    return text

def clear_column(df, column):
    df[column] = df[column].apply(clear_text)

def clear_columns(df, columns):
    pbar_conf = {
        'refresh_rate': 1,
        'action_names': columns
    }
        
    for c in Pbar(columns, **pbar_conf):
        clear_column(df, c)

In [118]:
clear_columns(df, ['title', 'perex', 'body'])

[                                                  ] (processing: title) -- 0 / 3

C:\Users\j\Anaconda3\lib\site-packages\bs4\__init__.py:314: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


[==================================                ] (processing: body) -- 2 / 33

C:\Users\j\Anaconda3\lib\site-packages\bs4\__init__.py:389: UserWarning: "http://medicalxpress.com/news/2013-08-vaccination-flu-worse-exposed-strain.html
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


[==================================================] -- 3 / 3 -- (finished)


In [244]:
label_names = list(filter(lambda x: x.endswith('_label'), df.columns))

In [245]:
for ln in label_names:
    df[ln] = pd.to_numeric(df[ln])

In [246]:
labels_df = pd.DataFrame()

In [247]:
# labely
labels_df = pd.concat([labels_df] + [df[label_name] for label_name in label_names], axis=1)

In [248]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164221 entries, 0 to 171743
Data columns (total 28 columns):
id                         164221 non-null int64
url                        164221 non-null object
title                      164221 non-null object
perex                      164221 non-null object
body                       164221 non-null object
published_at               164221 non-null datetime64[ns]
extracted_at               164221 non-null datetime64[ns]
source_id                  164221 non-null int64
category                   136391 non-null object
other_info                 164217 non-null object
image_count                164221 non-null int64
video_count                164221 non-null int64
author_name                164221 non-null object
source_id                  164221 non-null int64
source_name                164221 non-null object
source_url                 164221 non-null object
source_type                164221 non-null object
source_is_reliable         1

## Rozdelenie dat

In [252]:
train, test, validation = tuple(split_data(df, sizes=[2, 2, 1], shuffle=True, np_random=np_random))

In [253]:
print([len(i) for i in [train,test,validation]])

[65689, 65688, 32844]


In [254]:
label_names

['fb_reaction_count_label',
 'fb_comment_count_label',
 'fb_share_count_label',
 'fb_popularity_label',
 'is_fake_news_label']

---

In [217]:
from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob

import spacy

nlp = spacy.load("en_core_web_sm")

In [218]:
def tokenize(text):
    doc = nlp(text, disable=['parser', 'tagger', 'ner'])
    
    res = []
    for i in doc:
        if i.is_stop:
            continue
        if i.is_punct:
            continue
            
        res.append(str(i))
    
    return res

In [130]:
def title_basic_features(df):
    cv = CountVectorizer()
    data = cv.fit_transform(df.title)

    res = pd.DataFrame(index=df.index)
    
    res['title_word_count'] = data.sum(axis=1)
    res['title_char_length'] = df.title.apply(lambda x: len(x))
    
    return res

In [131]:
def perex_basic_features(df):
    cv = CountVectorizer()
    data = cv.fit_transform(df.perex)

    res = pd.DataFrame(index=df.index)    
    res['perex_word_count'] = data.sum(axis=1)
    res['perex_char_length'] = df.perex.apply(lambda x: len(x))
    
    return res

In [132]:
def content_basic_features(df):
    content_cv = CountVectorizer()
    data = content_cv.fit_transform(df.body)

    res = pd.DataFrame(index=df.index)    
    res['content_word_count'] = data.sum(axis=1)
    res['content_char_length'] = df.body.apply(lambda x: len(x))
    
    return res

In [133]:
def media_count_total(df):
    res = pd.DataFrame(index=df.index)
    
    res['media_count_total'] = df['image_count'] + df['video_count']
    
    return res
    
def media_count_image(df):
    return column_feature(df, 'image_count')

def media_count_video(df):
    return column_feature(df, 'video_count')

In [134]:
def published_on_day(df):
    res = pd.DataFrame(index=df.index)
    
    res['published_on_day'] = df.published_at.dt.weekday + 1
    
    return res

In [135]:
def is_collective_author(df):
    
    uniq_source_names = df.source_name.unique()
    def make_a_guess(author_name):

        return any((
                    str_contains(author_name, 'admin', case=False),
                    author_name.startswith('Neuroscience News Posts Science Research News Labs Universities Hospitals News Departments Around The World'),
                    author_name in ['Neuroscience News',
                                    'Wake Up World',
                                    'Health Sciences Institute',
                                    'REALdeal', 
                                    'nmheditor',
                                    'The Mind Unleashed',
                                    'Thinking Moms\' Revolution',
                                    'TheNewsDoctors',
                                    'clnews',
                                    'Associated Press',
                                    'HealthDay',
                                    'Infowars',
                                    'Natural News Editors',
                                    'https://www.facebook.com/WebMD',
                                    'naturalnews', 'peakconsciousness', 'HealingwithoutHurting',
                                    'HealthNutNews.com',
                                   ],
                    author_name.startswith('The Associated Press'),
                    # ' and ' in author_name, # todo: je to kolektivny autor ak ich je len viac?
                    author_name in uniq_source_names,   
        ))
    
    res = pd.DataFrame(index=df.index)
    
    res['is_collective_author'] = df.author_name.map(make_a_guess)
    
    return res

In [136]:
pd.Series(df.category.explode().unique())

0                                     Health
1                              Mental Health
2                                       News
3                                    Science
4                                      Weird
                        ...                 
2631    a5651504-8f5f-5320-9d04-cea5db37ce62
2632    2c63c2d2-7e9f-5e20-9a8d-8dc8a0c269fa
2633    1c36abf6-c842-5ef4-9952-12d650ea5c82
2634    937af7f5-414c-52ee-b01f-6a49488e9d52
2635    e8834c5b-20a3-517d-a488-32652a2a690c
Length: 2636, dtype: object

In [137]:
features = [
    title_basic_features,
    perex_basic_features,
    content_basic_features,
    
    media_count_total,
    media_count_image,
    media_count_video,
    
    published_on_day,
    is_collective_author,
]

In [265]:
def add_features(df):    
    pbar_conf = {
        'refresh_rate': 1,
        'action_names': [i.__name__ for i in features]
    }
    
    res = pd.DataFrame()
    for feature_generator in Pbar(features, **pbar_conf):
        res = pd.concat([res, feature_generator(df)], axis=1)

    return res

---

In [277]:
data = split_X_y_all(train, test, validation, selected_label='is_fake_news_label', all_labels=label_names)

In [287]:
data.train.features = add_features(data.train.X)

[==================================================] -- 8 / 8 -- (finished)tive_author) -- 7 / 8 8


In [288]:
data.test.features = add_features(data.test.X)

[==================================================] -- 8 / 8 -- (finished)tive_author) -- 7 / 8 8


In [289]:
data.validation.features = add_features(data.validation.X)

[==================================================] -- 8 / 8 -- (finished)tive_author) -- 7 / 8 8


In [276]:
label_names

['fb_reaction_count_label',
 'fb_comment_count_label',
 'fb_share_count_label',
 'fb_popularity_label',
 'is_fake_news_label']

In [335]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument


import multiprocessing

In [355]:
def tokenize_to_file(data, file):
    with open(file, 'w', encoding='utf-8') as f:
        for i in Pbar(data):
            f.write(f"{' '.join(tokenize(i))}\n")

In [327]:
cores = multiprocessing.cpu_count()
print(f'>>> {cores} cores available')

>>> 8 cores available


In [332]:
tokenize_to_file(data.train.X.body, './data/train_body_tokenized.txt)
tokenize_to_file(data.test.X.body, './data/test_body_tokenized.txt')
tokenize_to_file(data.validation.X.body, './data/validation_body_tokenized.txt')

[==================================================] -- 65689 / 65689 -- (finished)


In [337]:
d2v = Doc2Vec(corpus_file='./data/train_body_tokenized.txt', vector_size=300, min_count=2, epochs=15, workers=cores)

2020-03-24 23:28:50,335 : INFO : collecting all words and their counts
2020-03-24 23:28:50,337 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-03-24 23:28:51,590 : INFO : PROGRESS: at example #10000, processed 4298665 words (3432772/s), 123951 word types, 10000 tags
2020-03-24 23:28:52,876 : INFO : PROGRESS: at example #20000, processed 8568952 words (3324739/s), 183544 word types, 20000 tags
2020-03-24 23:28:54,350 : INFO : PROGRESS: at example #30000, processed 12895290 words (2935639/s), 233597 word types, 30000 tags
2020-03-24 23:28:55,908 : INFO : PROGRESS: at example #40000, processed 17179182 words (2751366/s), 278031 word types, 40000 tags
2020-03-24 23:28:58,066 : INFO : PROGRESS: at example #50000, processed 21473433 words (1990505/s), 319044 word types, 50000 tags
2020-03-24 23:29:00,432 : INFO : PROGRESS: at example #60000, processed 25781908 words (1821971/s), 357488 word types, 60000 tags
2020-03-24 23:29:01,813 : INFO : collected 3786

In [378]:
def infer_d2v(d2v_model, data_file):
    
    res = []
    
    with open(data_file, 'r', encoding='utf-8') as f:
        for i in Pbar(f.readlines()):
            res.append(d2v_model.infer_vector(i.split(' '), steps=20, alpha=0.025)) 
    
    return res

def infer_for_df(df, d2v_model, data_file):
    lst = infer_d2v(d2v_model, data_file)
    d2v_df = pd.DataFrame(lst, index=df.index, columns=[f'd2v_{i}' for i in range(1, 301)] )
    
    return pd.concat([df, d2v_df], axis=1, sort=False)

In [ ]:
data.train.features = infer_for_df(data.train.features, d2v, './data/train_body_tokenized.txt')

In [380]:
data.test.features = infer_for_df(data.test.features, d2v, './data/test_body_tokenized.txt')

[==================================================] -- 65688 / 65688 -- (finished)


In [394]:
from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [290]:
def fit_predict(clf, data):
    clf.fit(data.train.features, data.train.y)
    return clf.predict(data.test.features)

In [397]:
data.train.features['media_count_total'] = pd.to_numeric(data.train.features['media_count_total'])
data.train.features['image_count'] = pd.to_numeric(data.train.features['image_count'])
data.train.features['video_count'] = pd.to_numeric(data.train.features['video_count'])

data.test.features['media_count_total'] = pd.to_numeric(data.test.features['media_count_total'])
data.test.features['image_count'] = pd.to_numeric(data.test.features['image_count'])
data.test.features['video_count'] = pd.to_numeric(data.test.features['video_count'])

In [405]:
classifiers = [
    RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=cores),
    XGBClassifier(n_jobs=cores, seed=RANDOM_STATE),
]

pbar_conf = {
    'refresh_rate': 1,
    'length': len(classifiers), 
    'pbar_width': 52,
    'action_names': [i.__class__.__name__ for i in classifiers]
}

predictions = list(Pbar((fit_predict(clf, data) for clf in classifiers), **pbar_conf))

for p in predictions:
    print(classification_report(data.test.y, p))
    print('-' * 54)

[====================================================] -- 2 / 2 -- (finished)fier) -- 1 / 2 -- 0 / 2
              precision    recall  f1-score   support

           1       0.57      0.97      0.71     32588
           2       0.48      0.26      0.34     16658
           3       0.48      0.06      0.10      9928
           4       0.24      0.00      0.00      3302
           5       0.40      0.00      0.00      3212

    accuracy                           0.55     65688
   macro avg       0.43      0.26      0.23     65688
weighted avg       0.51      0.55      0.46     65688

------------------------------------------------------
              precision    recall  f1-score   support

           1       0.57      0.96      0.72     32588
           2       0.47      0.26      0.34     16658
           3       0.43      0.09      0.15      9928
           4       0.00      0.00      0.00      3302
           5       0.29      0.00      0.00      3212

    accuracy                 

In [407]:
display_all(pd.DataFrame((i for i in classifiers[0].feature_importances_), index=data.train.features.columns, columns=['importance']).sort_values(by=['importance'], ascending=False))

,importance
perex_word_count,0.011719
perex_char_length,0.011410
content_char_length,0.008882
content_word_count,0.007891
title_char_length,0.007131
title_word_count,0.004839
media_count_total,0.004458
image_count,0.004145
d2v_287,0.003971
is_collective_author,0.003797


In [ ]:
show_importances(classifiers[0], data.train.features.columns)

In [385]:
data.train.y

0        1
1        0
2        1
3        1
4        1
        ..
65684    1
65685    1
65686    0
65687    1
65688    1
Name: is_fake_news_label, Length: 65689, dtype: int64